In [1]:
from prefect import task
import requests
import pandas as pd
import numpy as np
import sqlalchemy
from decouple import config 
import psycopg2


def hn_scrape(i,comments_list):
    r = requests.get('https://hacker-news.firebaseio.com/v0/item/'+str(i)+'.json').json()
    try:
        if ('deleted' in r.keys()):
            pass
        else:
            if r["type"] == 'comment':
                t = (r["by"],r["id"],r["text"],r["time"])
                comments_list.append(t)
    except:
        pass

def sentiment(text):
    payload = {'text':text}
    try:
        return requests.get('https://crawftv-nlp-api.herokuapp.com/sentiment',params=payload).json()["compound"]
    except:
        pass
    

@task
def extract():
    comments_list = []
    conn = db.connect()
    curs = conn.connection.cursor()
    curs.execute(" SELECT MAX(id) FROM comments")
    table_max = curs.fetchall()
    table_max = int(table_max[0][0])
    conn.close()
    curs.close()
    for i in range(table_max,table_max+10000,1):
        hn_scrape(i,comments_list)
    return comments_list
    
    

@task
def transform(comments_list):
    comments= pd.DataFrame(columns = ["by", "id","text","time"], data=comments_list)
    comments["text"] = comments["text"].str.replace("&quot;","")
    comments["text"] = comments["text"].str.replace("&#x27;","'")
    comments["text"] = comments["text"].str.replace("&gt; "," ")
    comments["text"] = comments["text"].str.replace("<p>"," ")
    comments["text"] = comments["text"].str.replace("<a>"," ")
    comments["text"] = comments["text"].str.replace("</a>"," ")
    comments["text"] = comments["text"].str.replace("<i>"," ")
    comments["text"] = comments["text"].str.replace("</i>"," ")
    comments["text"] = comments["text"].str.replace("&#x2F;",'')
    comments["text"] = comments["text"].str.replace("https:",' ')
    comments["text"] = comments["text"].str.replace("\\n",' ')
    comments = comments.dropna()
    return comments


@task
def load(comments):
    comments["sentiment"] = comments["text"].apply(sentiment)
    conn = db.connect()
    curs = conn.connection.cursor()
    comments.to_sql(name="comments",con=db, if_exists="append",chunksize=500)
    conn.close()
    curs.close()

In [2]:
from prefect import Flow
host = config('AWS_DATABASE_URL')
db = sqlalchemy.create_engine(host)


with Flow('ETL') as flow:
    e = extract()
    t = transform(e)
    l = load(t)

flow.run()

[2019-06-19 16:21:06,764] INFO - prefect.FlowRunner | Beginning Flow run for 'ETL'
[2019-06-19 16:21:06,765] INFO - prefect.FlowRunner | Starting flow run.
[2019-06-19 16:21:06,768] INFO - prefect.TaskRunner | Task 'extract': Starting task run...
[2019-06-19 16:41:39,212] INFO - prefect.TaskRunner | Task 'extract': finished task run for task with final state: 'Success'
[2019-06-19 16:41:39,213] INFO - prefect.TaskRunner | Task 'transform': Starting task run...
[2019-06-19 16:41:39,306] INFO - prefect.TaskRunner | Task 'transform': finished task run for task with final state: 'Success'
[2019-06-19 16:41:39,308] INFO - prefect.TaskRunner | Task 'load': Starting task run...
[2019-06-19 17:11:47,317] INFO - prefect.TaskRunner | Task 'load': finished task run for task with final state: 'Success'
[2019-06-19 17:11:47,319] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">